# Feed-forward Neural Networks

This note book will be used to test Feed-forward neural networks on each of the 3 ontologies.

In [10]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
import itertools
import yaml

In [2]:
print("TensorFlow v" + tf.__version__)
print("Numpy v" + np.__version__)

TensorFlow v2.15.0
Numpy v1.26.4


In [3]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [3]:
config_path = './config.yaml'
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

In [4]:
USE_75_PERCENT_DATA = config['use_75_percent_datasets']
partial_dataset_prefix = '75percent_' if USE_75_PERCENT_DATA else ''
BP_train_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_embeddings_BiologicalProcesses.pkl")
CC_train_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_embeddings_CellularComponent.pkl")
MF_train_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_embeddings_MolecularFunction.pkl")
BP_label_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_labels_BiologicalProcesses.pkl")
CC_label_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_labels_CellularComponent.pkl")
MF_label_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_labels_MolecularFunction.pkl")

In [5]:
train_data_dict = {'Biological Processes': [BP_train_df, BP_label_df],
                   'Cellular Component': [CC_train_df, CC_label_df],
                   'Molecular Function': [MF_train_df, MF_label_df]
}

In [6]:
num_labels = 1500
num_folds = config['num_folds']

## Model 1 Architecture: All Dense layers of the same size

In [7]:
BATCH_SIZE = config['batch_size']

In [8]:
def model1_training(dataset_name, data):
  train = data[0]
  label = data[1]

  INPUT_SHAPE = [train.shape[1]]

  best_f1 = 0
  print('=======================================================================')
  print(f'Training for {dataset_name}')
  
  model_root_path = f'{config["directories"]["models"]}/FFNNMod1'

  for unit_param, act_param in itertools.product([512, 256, 128], ['relu', 'tanh']):
    print('----------------------------------------------------------------------')
    print('The number of units in each layer is ', unit_param)
    print('The activation function in each layer is ', act_param)

    kfold = KFold(n_splits=num_folds, shuffle=True)
    fold_no = 1

    for train_fold, test_fold in kfold.split(train, label):

      model1 = tf.keras.Sequential([
          tf.keras.layers.BatchNormalization(input_shape=INPUT_SHAPE),
          tf.keras.layers.Dense(units = unit_param, activation = act_param),
          tf.keras.layers.Dense(units = unit_param, activation = act_param),
          tf.keras.layers.Dense(units = unit_param, activation = act_param),
          tf.keras.layers.Dense(units = num_labels, activation = 'sigmoid')
          ])

      # Compile model
      model1.compile(
          optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
          loss='binary_crossentropy',
          metrics=['binary_accuracy',
                  tf.keras.metrics.AUC(),
                  tf.keras.metrics.Precision(),
                  tf.keras.metrics.Recall(),
                  ] # tf.keras.metrics.F1Score() not appropriate as it is calculated batchwise
          )

      print(f'Training for fold {fold_no} ...')

      # Fit the data to the model
      history = model1.fit(
            train.iloc[train_fold], label.iloc[train_fold],
            validation_data=(train.iloc[test_fold], label.iloc[test_fold]),
            batch_size=BATCH_SIZE,
            epochs=5
        )

      # Generate metrics
      scores = model1.evaluate(train.iloc[test_fold], label.iloc[test_fold], verbose=0)
      precision = scores[3]
      recall = scores[4]
      F1_score = 2*precision*recall / (precision + recall)
      print(f'Score for fold {fold_no}: F1 score of {F1_score}; {model1.metrics_names[0]} of {scores[0]}; {model1.metrics_names[1]} of {scores[1]*100}%')

      if F1_score > best_f1:
        best_f1 = F1_score
        if dataset_name == 'Biological Processes':
          tf.keras.models.save_model(
              model1,
              f'{model_root_path}/best_BP_model',
          )
          print(f'Current best model for Biological Processes has {unit_param} units in each layer, uses {act_param} activation function and has an F1 score of {F1_score}')

        elif dataset_name == 'Molecular Function':
          tf.keras.models.save_model(
              model1,
              f'{model_root_path}/best_MF_model',
          )
          print(f'Current best model for Molecular Function has {unit_param} units in each layer, uses {act_param} activation function and has an F1 score of {F1_score}')

        else:
          tf.keras.models.save_model(
              model1,
              f'{model_root_path}/best_CC_model',
          )
          print(f'Current best model for Cellular Component has {unit_param} units in each layer, uses {act_param} activation function and has an F1 score of {F1_score}')

      fold_no += 1

In [ ]:
for dataset in train_data_dict:
  dataset_name = dataset
  data = train_data_dict[dataset]
  model1_training(dataset_name, data)

## Model 2 Architecture: Decreasing then increasing layer size

In [11]:
BATCH_SIZE = config['batch_size']

In [12]:
def model2_training(dataset_name, data):
  train = data[0]
  label = data[1]

  INPUT_SHAPE = [train.shape[1]]

  best_f1 = 0
  print('=======================================================================')
  print(f'Training for {dataset_name}')

  model_root_path = f'{config["directories"]["models"]}/FFNNMod2'

  act_param = 'relu'

  kfold = KFold(n_splits=num_folds, shuffle=True)
  fold_no = 1

  for train_fold, test_fold in kfold.split(train, label):

    model2 = tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(input_shape=INPUT_SHAPE),
        tf.keras.layers.Dense(units = 512, activation = act_param),
        tf.keras.layers.Dense(units = 256, activation = act_param),
        tf.keras.layers.Dense(units = 128, activation = act_param),
        tf.keras.layers.Dense(units = 256, activation = act_param),
        tf.keras.layers.Dense(units = 512, activation = act_param),
        tf.keras.layers.Dense(units = num_labels, activation = 'sigmoid')
        ])

    # Compile model
    model2.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['binary_accuracy',
                tf.keras.metrics.AUC(),
                tf.keras.metrics.Precision(),
                tf.keras.metrics.Recall(),
                ] # tf.keras.metrics.F1Score() not appropriate as it is calculated batchwise
        )

    print(f'Training for fold {fold_no} ...')

    # Fit the data to the model
    history = model2.fit(
            train.iloc[train_fold], label.iloc[train_fold],
            validation_data=(train.iloc[test_fold], label.iloc[test_fold]),
            batch_size=BATCH_SIZE,
            epochs=5
        )

    # Generate metrics
    scores = model2.evaluate(train, label, verbose=0)
    precision = scores[3]
    recall = scores[4]
    F1_score = 2*precision*recall / (precision + recall)
    print(f'Score for fold {fold_no}: F1 score of {F1_score}; {model2.metrics_names[0]} of {scores[0]}; {model2.metrics_names[1]} of {scores[1]*100}%')

    if F1_score > best_f1:
      best_f1 = F1_score
      if dataset_name == 'Biological Processes':
        tf.keras.models.save_model(
            model2,
            f'{model_root_path}/best_BP_model',
        )
        print(f'Current best model for Biological Processes has an F1 score of {F1_score}')

      elif dataset_name == 'Molecular Function':
        tf.keras.models.save_model(
            model2,
            f'{model_root_path}/best_MF_model',
        )
        print(f'Current best model for Molecular Function has an F1 score of {F1_score}')

      else:
        tf.keras.models.save_model(
            model2,
            f'{model_root_path}/best_CC_model',
        )
        print(f'Current best model for Cellular Component has an F1 score of {F1_score}')

    fold_no += 1

In [13]:
for dataset in train_data_dict:
  dataset_name = dataset
  data = train_data_dict[dataset]
  model2_training(dataset_name, data)

Training for Biological Processes
Training for fold 1 ...
Epoch 1/5
293/293 [==============================] - 6s 7ms/step - loss: 0.1038 - binary_accuracy: 0.9698 - auc_180: 0.8562 - precision_180: 0.4178 - recall_180: 0.1512 - val_loss: 0.2583 - val_binary_accuracy: 0.9744 - val_auc_180: 0.9121 - val_precision_180: 0.6870 - val_recall_180: 0.1788
Epoch 2/5
293/293 [==============================] - 2s 7ms/step - loss: 0.0798 - binary_accuracy: 0.9748 - auc_180: 0.9201 - precision_180: 0.6757 - recall_180: 0.2247 - val_loss: 0.0854 - val_binary_accuracy: 0.9750 - val_auc_180: 0.9270 - val_precision_180: 0.6852 - val_recall_180: 0.2143
Epoch 3/5
293/293 [==============================] - 2s 6ms/step - loss: 0.0762 - binary_accuracy: 0.9753 - auc_180: 0.9295 - precision_180: 0.6821 - recall_180: 0.2543 - val_loss: 0.0780 - val_binary_accuracy: 0.9747 - val_auc_180: 0.9259 - val_precision_180: 0.6252 - val_recall_180: 0.2663
Epoch 4/5
293/293 [==============================] - 2s 6ms/ste

INFO:tensorflow:Assets written to: ./models/FFNNMod2/best_BP_model/assets


Current best model for Biological Processes has an F1 score of 0.40463973992582647
Training for fold 2 ...
Epoch 1/5
293/293 [==============================] - 5s 7ms/step - loss: 0.1025 - binary_accuracy: 0.9699 - auc_181: 0.8606 - precision_181: 0.4241 - recall_181: 0.1580 - val_loss: 0.2490 - val_binary_accuracy: 0.9741 - val_auc_181: 0.9141 - val_precision_181: 0.7060 - val_recall_181: 0.1706
Epoch 2/5
293/293 [==============================] - 2s 5ms/step - loss: 0.0794 - binary_accuracy: 0.9749 - auc_181: 0.9207 - precision_181: 0.6777 - recall_181: 0.2272 - val_loss: 0.0861 - val_binary_accuracy: 0.9745 - val_auc_181: 0.9246 - val_precision_181: 0.6972 - val_recall_181: 0.2038
Epoch 3/5
293/293 [==============================] - 1s 5ms/step - loss: 0.0761 - binary_accuracy: 0.9754 - auc_181: 0.9297 - precision_181: 0.6820 - recall_181: 0.2540 - val_loss: 0.0777 - val_binary_accuracy: 0.9750 - val_auc_181: 0.9259 - val_precision_181: 0.6650 - val_recall_181: 0.2643
Epoch 4/5
293/

INFO:tensorflow:Assets written to: ./models/FFNNMod2/best_BP_model/assets


Current best model for Biological Processes has an F1 score of 0.4313931855785713
Training for fold 6 ...
Epoch 1/5
293/293 [==============================] - 4s 6ms/step - loss: 0.1038 - binary_accuracy: 0.9698 - auc_185: 0.8566 - precision_185: 0.4162 - recall_185: 0.1494 - val_loss: 0.2500 - val_binary_accuracy: 0.9743 - val_auc_185: 0.9130 - val_precision_185: 0.6760 - val_recall_185: 0.1918
Epoch 2/5
293/293 [==============================] - 2s 5ms/step - loss: 0.0798 - binary_accuracy: 0.9748 - auc_185: 0.9195 - precision_185: 0.6759 - recall_185: 0.2237 - val_loss: 0.0845 - val_binary_accuracy: 0.9748 - val_auc_185: 0.9271 - val_precision_185: 0.7045 - val_recall_185: 0.2038
Epoch 3/5
293/293 [==============================] - 2s 5ms/step - loss: 0.0763 - binary_accuracy: 0.9754 - auc_185: 0.9291 - precision_185: 0.6835 - recall_185: 0.2529 - val_loss: 0.0773 - val_binary_accuracy: 0.9751 - val_auc_185: 0.9250 - val_precision_185: 0.6789 - val_recall_185: 0.2459
Epoch 4/5
293/2

...

Current best model for Cellular Component has an F1 score of 0.4962278912594993
Training for fold 5 ...
Epoch 1/5
298/298 [==============================] - 4s 7ms/step - loss: 0.0892 - binary_accuracy: 0.9747 - auc_194: 0.8712 - precision_194: 0.4729 - recall_194: 0.2075 - val_loss: 0.2440 - val_binary_accuracy: 0.9792 - val_auc_194: 0.9239 - val_precision_194: 0.7534 - val_recall_194: 0.2359
Epoch 2/5
298/298 [==============================] - 2s 5ms/step - loss: 0.0673 - binary_accuracy: 0.9795 - auc_194: 0.9273 - precision_194: 0.7452 - recall_194: 0.2627 - val_loss: 0.0754 - val_binary_accuracy: 0.9797 - val_auc_194: 0.9359 - val_precision_194: 0.7495 - val_recall_194: 0.2675
Epoch 3/5
298/298 [==============================] - 2s 5ms/step - loss: 0.0639 - binary_accuracy: 0.9800 - auc_194: 0.9370 - precision_194: 0.7423 - recall_194: 0.2912 - val_loss: 0.0644 - val_binary_accuracy: 0.9800 - val_auc_194: 0.9358 - val_precision_194: 0.7338 - val_recall_194: 0.2987
Epoch 4/5
298/298

INFO:tensorflow:Assets written to: ./models/FFNNMod2/best_MF_model/assets


Current best model for Molecular Function has an F1 score of 0.48580617788975133
Training for fold 2 ...
Epoch 1/5
196/196 [==============================] - 4s 9ms/step - loss: 0.1153 - binary_accuracy: 0.9666 - auc_201: 0.8434 - precision_201: 0.3965 - recall_201: 0.1722 - val_loss: 0.3746 - val_binary_accuracy: 0.9731 - val_auc_201: 0.9100 - val_precision_201: 0.7056 - val_recall_201: 0.2238
Epoch 2/5
196/196 [==============================] - 1s 6ms/step - loss: 0.0827 - binary_accuracy: 0.9741 - auc_201: 0.9212 - precision_201: 0.7105 - recall_201: 0.2593 - val_loss: 0.1785 - val_binary_accuracy: 0.9740 - val_auc_201: 0.9295 - val_precision_201: 0.7143 - val_recall_201: 0.2656
Epoch 3/5
196/196 [==============================] - 1s 5ms/step - loss: 0.0776 - binary_accuracy: 0.9748 - auc_201: 0.9336 - precision_201: 0.7164 - recall_201: 0.2969 - val_loss: 0.0883 - val_binary_accuracy: 0.9747 - val_auc_201: 0.9373 - val_precision_201: 0.7187 - val_recall_201: 0.2995
Epoch 4/5
196/19

INFO:tensorflow:Assets written to: ./models/FFNNMod2/best_MF_model/assets


Current best model for Molecular Function has an F1 score of 0.4962166985258705
Training for fold 8 ...
Epoch 1/5
196/196 [==============================] - 50s 12ms/step - loss: 0.1137 - binary_accuracy: 0.9669 - auc_207: 0.8476 - precision_207: 0.4112 - recall_207: 0.1782 - val_loss: 0.3705 - val_binary_accuracy: 0.9737 - val_auc_207: 0.9139 - val_precision_207: 0.7092 - val_recall_207: 0.2281
Epoch 2/5
196/196 [==============================] - 1s 6ms/step - loss: 0.0826 - binary_accuracy: 0.9740 - auc_207: 0.9217 - precision_207: 0.7116 - recall_207: 0.2606 - val_loss: 0.1748 - val_binary_accuracy: 0.9746 - val_auc_207: 0.9333 - val_precision_207: 0.7019 - val_recall_207: 0.2861
Epoch 3/5
196/196 [==============================] - 1s 6ms/step - loss: 0.0777 - binary_accuracy: 0.9748 - auc_207: 0.9338 - precision_207: 0.7160 - recall_207: 0.2987 - val_loss: 0.0875 - val_binary_accuracy: 0.9753 - val_auc_207: 0.9394 - val_precision_207: 0.7175 - val_recall_207: 0.3111
Epoch 4/5
196/1